### Setup

In this section, we load necessary libraries and define custom functions.

In [273]:
# install PPI library if needed 
# %pip install git+https://github.com/Michael-Howes/ppi_py.git


In [274]:
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
import sys
from scipy import stats
from ppi_py import ppi_ols_ci, classical_ols_ci, ppi_ols_pointestimate

df = pd.read_csv("../Data/5_SurveySampleLLM.csv.gz")

Covs = ['PedPed', 'Barrier', 'CrossingSignal', 'NumberOfCharacters',
        'DiffNumberOFCharacters', 'LeftHand', 'Man', 'Woman', 'Pregnant',
        'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
        'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
        'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
        'MaleDoctor', 'Dog', 'Cat', 
        'Intervention'
        ]

sys.version

'3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]'

In [275]:
print("Number of respondents: ", len(df["UserID"].unique()))
print("Number of decisions: ", len(df["ResponseID"].unique()))
print("Number of NAs in observed dependent variable: ", df["Saved"].isna().sum())
print("Number of NAs in predicted dependent variable with GPT4 Turbo: ", df["gpt4turbo_wp_Saved_1"].isna().sum())

Number of respondents:  55893
Number of decisions:  581981
Number of NAs in observed dependent variable:  0
Number of NAs in predicted dependent variable with GPT4 Turbo:  6694


### Reproduce AMCE from R functions

Awad et al. (2018) use R to estimate the AMCE for the the conjoint experiment. In this section, we verify that we can obtain the results with our Python code.

In [276]:
def CalcTheoreticalInt(r):
    # this function is applied to each row (r)
    if r["Intervention"]==0:
        if r["Barrier"]==0:
            if r["PedPed"]==1: p = 0.48
            else: p = 0.32
            
            if r["CrossingSignal"]==0:   p = p * 0.48
            elif r["CrossingSignal"]==1: p = p * 0.2
            else: p = p * 0.32
        else: p = 0.2

    else: 
        if r["Barrier"]==0:
            if r["PedPed"]==1: 
                p = 0.48
                if r["CrossingSignal"]==0: p = p * 0.48
                elif r["CrossingSignal"]==1: p = p * 0.32
                else: p = p * 0.2
            else: 
                p = 0.2
                if r["CrossingSignal"]==0: p = p * 0.48
                elif r["CrossingSignal"]==1: p = p * 0.2
                else: p = p * 0.32
        else: p = 0.32  
    
    return(p)  
        
def calcWeightsTheoretical(profiles):
    
    p = profiles.apply(CalcTheoreticalInt, axis=1)

    weight = 1/p 

    return(weight)         

In [277]:
# function from PPI to calculate stats
def _ols_get_stats(
    pointest,
    X,
    Y,
    Yhat,
    X_unlabeled,
    Yhat_unlabeled,
    w=None,
    w_unlabeled=None,
    use_unlabeled=True,
):
    """Computes the statistics needed for the OLS-based prediction-powered inference.

    Args:
        pointest (ndarray): A point estimate of the coefficients.
        X (ndarray): Covariates for the labeled data set.
        Y (ndarray): Labels for the labeled data set.
        Yhat (ndarray): Predictions for the labeled data set.
        X_unlabeled (ndarray): Covariates for the unlabeled data set.
        Yhat_unlabeled (ndarray): Predictions for the unlabeled data set.
        w (ndarray, optional): Sample weights for the labeled data set.
        w_unlabeled (ndarray, optional): Sample weights for the unlabeled data set.
        use_unlabeled (bool, optional): Whether to use the unlabeled data set.

    Returns:
        grads (ndarray): Gradient of the loss function with respect to the coefficients.
        grads_hat (ndarray): Gradient of the loss function with respect to the coefficients, evaluated using the labeled predictions.
        grads_hat_unlabeled (ndarray): Gradient of the loss function with respect to the coefficients, evaluated using the unlabeled predictions.
        inv_hessian (ndarray): Inverse Hessian of the loss function with respect to the coefficients.
    """
    n = Y.shape[0]
    N = Yhat_unlabeled.shape[0]
    d = X.shape[1]
    w = np.ones(n) if w is None else w / np.sum(w) * n
    w_unlabeled = (
        np.ones(N)
        if w_unlabeled is None
        else w_unlabeled / np.sum(w_unlabeled) * N
    )

    hessian = np.zeros((d, d))
    grads_hat_unlabeled = np.zeros(X_unlabeled.shape)
    if use_unlabeled:
        for i in range(N):
            hessian += (
                w_unlabeled[i]
                / (N + n)
                * np.outer(X_unlabeled[i], X_unlabeled[i])
            )
            grads_hat_unlabeled[i, :] = (
                w_unlabeled[i]
                * X_unlabeled[i, :]
                * (np.dot(X_unlabeled[i, :], pointest) - Yhat_unlabeled[i])
            )

    grads = np.zeros(X.shape)
    grads_hat = np.zeros(X.shape)
    for i in range(n):
        hessian += (
            w[i] / (N + n) * np.outer(X[i], X[i])
            if use_unlabeled
            else w[i] / n * np.outer(X[i], X[i])
        )
        grads[i, :] = w[i] * X[i, :] * (np.dot(X[i, :], pointest) - Y[i])
        grads_hat[i, :] = (
            w[i] * X[i, :] * (np.dot(X[i, :], pointest) - Yhat[i])
        )

    inv_hessian = np.linalg.inv(hessian).reshape(d, d)
    return grads, grads_hat, grads_hat_unlabeled, inv_hessian

def _power_analysis_stats(grads, grads_hat, inv_hessian):
    grads_ = grads - grads.mean(axis=0)
    grads_hat_ = grads_hat - grads_hat.mean(axis=0)
    cov = inv_hessian @ (grads_[:,None,:] * grads_hat_[:,:,None]).mean(axis=0) @ inv_hessian
    var = inv_hessian @ (grads_[:,None,:]*grads_[:,:,None]).mean(axis=0) @ inv_hessian
    var_hat = inv_hessian @ (grads_hat_[:,None,:]*grads_hat_[:,:,None]).mean(axis=0) @ inv_hessian
    rhos_sq = np.diag(cov)**2/(np.diag(var)*np.diag(var_hat))
    sigmas_sq = np.diag(var)
    return rhos_sq, sigmas_sq

def _estimate_ppi_SE(n, N, rho_sq, var_Y):
    if N == np.inf:
        return np.sqrt(var_Y*(1-rho_sq)/n)
    if N == 0:
        return np.sqrt(var_Y/n)
    var_ppi = var_Y*(1-rho_sq*N/(n+N))/n
    return np.sqrt(var_ppi)

def _estimate_classical_SE(n, var_Y):
    return np.sqrt(var_Y/n)

Below we define a function to compute the Average Marginal Component Effect (AMCE) for an attribute of the moral dilemmas using  weighted least squares. 

In [278]:
def compute_amce(data, x, y, alpha=0.05):

    # specify regression for swerve or stay in lane
    if x=="Intervention":
        
        # calculate weights
        data.loc[:,"weights"] = calcWeightsTheoretical(data)
    
        # drop rows with missing values on dependent variable
        dd = data.dropna(subset=y)

        # if X=1 characters die if AV serves, if X=0 characters if AV stays
        X = dd["Intervention"]
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])
    

    # specify regression for relationship to vehicle
    if x=="Barrier":

        # consider only dilemmas without legality and only pedestrians vs passengers
        data_sub = data.loc[(data["CrossingSignal"]==0) & (data["PedPed"]==0), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        
        # if X=1 passengers die and if X=0 pedestrians die
        X = dd["Barrier"]

        # recode to estimate the preference for pedestrians over passengers 
        X = 1 - X
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    
    # specify regression for legality
    if x=="CrossingSignal": 
        
        # consider dilemmas with legality and only pedestrians vs pedestrians
        data_sub = data.loc[(data["CrossingSignal"]!=0) & (data["PedPed"]==1), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)

        # if X=1 pedestrians cross on a green light, if X=2 pedestrians cross on a red light 
        X = dd["CrossingSignal"]

        # create dummy variable to estimate preference for pedestrians that cross legally (1) vs legally (0)
        X = 2 - X 
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    

    # Specify regressions for the remaining six attributes
    if x=="Utilitarian":
        
        # consider dilemmas that compare 'More' versus 'Less' characters
        data_sub = data.loc[(data["ScenarioType"]=="Utilitarian") & (data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Utilitarian'})

        # create dummy variable to estimate the preference for sparing more characters
        X = (dd.loc[:,"Utilitarian"]=="More").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Species":
        
        # consider dilemmas that compare humans versus animals 
        data_sub = data.loc[(data["ScenarioType"]=="Species") & (data["ScenarioTypeStrict"]=="Species"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Species'})

        # create dummy variable to estimate the preference for sparing humans
        X = (dd.loc[:,"Species"]=="Hoomans").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])
    

    if x=="Gender":
        
        # consider dilemmas that compare women versus men
        data_sub = data.loc[(data["ScenarioType"]=="Gender") & (data["ScenarioTypeStrict"]=="Gender"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Gender'})

        # create dummy variable to estimate the preference for sparing women
        X = (dd.loc[:,"Gender"]=="Female").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Fitness":
        
        # consider dilemmas that compare fit characters versus those that are not
        data_sub = data.loc[(data["ScenarioType"]=="Fitness") & (data["ScenarioTypeStrict"]=="Fitness"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Fitness'})

        # create dummy variable to estimate the preference for sparing fit characters
        X = (dd.loc[:,"Fitness"]=="Fit").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Age":
        
        # consider dilemmas that compare younger versus older characters
        data_sub = data.loc[(data["ScenarioType"]=="Age") & (data["ScenarioTypeStrict"]=="Age"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Age'})

        # create dummy variable to estimate the preference for sparing younger characters
        X = (dd.loc[:,"Age"]=="Young").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    
    if x=="Social Status":
        
        # consider dilemmas that compare high status versus low status characters
        data_sub = data.loc[(data["ScenarioType"]=="Social Status") & (data["ScenarioTypeStrict"]=="Social Status"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Social Status'})

        # create dummy variable to estimate the preference for sparing high status characters
        X = (dd.loc[:,"Social Status"]=="High").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])



    # fit model and extract estimates
    fit = model.fit(cov_type = 'cluster', cov_kwds = {'groups': dd["UserID"]})
    coef = fit.params[x]
    se = fit.bse[x]
    ci = fit.conf_int(alpha=alpha).loc[x]

    # store results
    res = pd.DataFrame({
        'x': [x],
        'y': [y],
        'beta': [coef],
        'se': [se],
        'lower': [ci[0]],
        'upper': [ci[1]]
    })

    return(res)


First, we compute the AMCEs only with data from human subjects using the functions defined above.

In [279]:
amce_human_subjects = pd.concat([
    compute_amce(df, x="Intervention", y="Saved"), 
    compute_amce(df, x="Barrier", y="Saved"), 
    compute_amce(df, x="Gender", y="Saved"), 
    compute_amce(df, x="Fitness", y="Saved"), 
    compute_amce(df, x="Social Status", y="Saved"), 
    compute_amce(df, x="CrossingSignal",y="Saved"),
    compute_amce(df, x="Age", y="Saved"),
    compute_amce(df, x="Utilitarian", y="Saved"),
    compute_amce(df, x="Species", y="Saved")
])      
amce_human_subjects.round(3)

,x,y,beta,se,lower,upper
0,Intervention,Saved,0.081,0.002,0.078,0.084
0,Barrier,Saved,0.105,0.003,0.100,0.111
0,Gender,Saved,0.135,0.003,0.129,0.142
0,Fitness,Saved,0.176,0.004,0.169,0.183
0,Social Status,Saved,0.240,0.009,0.221,0.258
0,CrossingSignal,Saved,0.377,0.003,0.372,0.383
0,Age,Saved,0.508,0.003,0.502,0.514
0,Utilitarian,Saved,0.571,0.003,0.565,0.576
0,Species,Saved,0.684,0.003,0.678,0.689


The AMCE estimates above are the same as those calculated with the functions by Awad et al. (2018), see object `main.Saved` in the R script `8_CalculateAMCE.R`. Hence, the custom functions defined in this notebook give the same results as the functions defined in the original article. 


|           label            |    dv  |  amce |   se  | conf.low | conf.high |
|----------------------------|--------|-------|-------|----------|-----------|
|   Intervention             | Saved  | 0.068 | 0.008 |    0.052 |     0.084 |
|        Barrier             | Saved  | 0.165 | 0.014 |    0.137 |     0.193 |
|            Law             | Saved  | 0.336 | 0.015 |    0.307 |     0.366 |
|         Gender             | Saved  | 0.160 | 0.017 |    0.127 |     0.193 |
|        Fitness             | Saved  | 0.121 | 0.018 |    0.085 |     0.156 |
|  Social Status             | Saved  | 0.171 | 0.047 |    0.079 |     0.263 |
|            Age             | Saved  | 0.482 | 0.016 |    0.451 |     0.513 |
| No. Characters             | Saved  | 0.573 | 0.014 |    0.545 |     0.602 |
|        Species             | Saved  | 0.646 | 0.015 |    0.617 |     0.675 |


In [280]:
def compute_amce_ppi(n_data, N_data, x, y, alpha=0.05):

    # specify regression for swerve or stay in lane
    if x=="Intervention":
        
        # calculate weights
        n_data.loc[:,"weights"] = calcWeightsTheoretical(n_data)
        N_data.loc[:,"weights"] = calcWeightsTheoretical(N_data)
    
        # drop rows with missing values on dependent variable
        n_dd = n_data.dropna(subset=y)
        N_dd = N_data.dropna(subset=y)

        # if X=1 characters die if AV serves, if X=0 characters if AV stays
        n_X = n_dd["Intervention"]               
        N_X = N_dd["Intervention"]

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    # specify regression for relationship to vehicle
    if x=="Barrier":

        # consider only dilemmas without legality and only pedestrians vs passengers
        n_data_sub = n_data.loc[(n_data["CrossingSignal"]==0) & (n_data["PedPed"]==0), :].copy()
        N_data_sub = N_data.loc[(N_data["CrossingSignal"]==0) & (N_data["PedPed"]==0), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)
        
        # if X=1 passengers die and if X=0 pedestrians die
        n_X = n_dd["Barrier"]
        N_X = N_dd["Barrier"]

        # recode to estimate the preference for pedestrians over passengers 
        n_X = 1 - n_X
        N_X = 1 - N_X

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights

    

    # specify regression for legality
    if x=="CrossingSignal": 
        
        # consider dilemmas with legality and only pedestrians vs pedestrians
        n_data_sub = n_data.loc[(n_data["CrossingSignal"]!=0) & (n_data["PedPed"]==1), :].copy()
        N_data_sub = N_data.loc[(N_data["CrossingSignal"]!=0) & (N_data["PedPed"]==1), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # if X=1 pedestrians cross on a green light, if X=2 pedestrians cross on a red light 
        n_X = n_dd["CrossingSignal"]
        N_X = N_dd["CrossingSignal"]

        # create dummy variable to estimate preference for pedestrians that cross legally (1) vs legally (0)
        n_X = 2 - n_X 
        N_X = 2 - N_X 

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights
    


    # Specify regressions for the remaining six attributes
    if x=="Utilitarian":
        
        # consider dilemmas that compare 'More' versus 'Less' characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Utilitarian") & (n_data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Utilitarian") & (N_data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)
        
        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Utilitarian'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Utilitarian'})

        # create dummy variable to estimate the preference for sparing more characters
        n_X = (n_dd.loc[:,"Utilitarian"]=="More").astype(int)
        N_X = (N_dd.loc[:,"Utilitarian"]=="More").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Species":
        
        # consider dilemmas that compare humans versus animals 
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Species") & (n_data["ScenarioTypeStrict"]=="Species"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Species") & (N_data["ScenarioTypeStrict"]=="Species"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Species'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Species'})

        # create dummy variable to estimate the preference for sparing humans
        n_X = (n_dd.loc[:,"Species"]=="Hoomans").astype(int)
        N_X = (N_dd.loc[:,"Species"]=="Hoomans").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights

    

    if x=="Gender":
        
        # consider dilemmas that compare women versus men
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Gender") & (n_data["ScenarioTypeStrict"]=="Gender"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Gender") & (N_data["ScenarioTypeStrict"]=="Gender"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Gender'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Gender'})

        # create dummy variable to estimate the preference for sparing women
        n_X = (n_dd.loc[:,"Gender"]=="Female").astype(int)
        N_X = (N_dd.loc[:,"Gender"]=="Female").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Fitness":
        
        # consider dilemmas that compare fit characters versus those that are not
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Fitness") & (n_data["ScenarioTypeStrict"]=="Fitness"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Fitness") & (N_data["ScenarioTypeStrict"]=="Fitness"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Fitness'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Fitness'})

        # create dummy variable to estimate the preference for sparing fit characters
        n_X = (n_dd.loc[:,"Fitness"]=="Fit").astype(int)
        N_X = (N_dd.loc[:,"Fitness"]=="Fit").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Age":
        
        # consider dilemmas that compare younger versus older characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Age") & (n_data["ScenarioTypeStrict"]=="Age"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Age") & (N_data["ScenarioTypeStrict"]=="Age"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Age'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Age'})

        # create dummy variable to estimate the preference for sparing younger characters
        n_X = (n_dd.loc[:,"Age"]=="Young").astype(int)
        N_X = (N_dd.loc[:,"Age"]=="Young").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights


    
    if x=="Social Status":
        
        # consider dilemmas that compare high status versus low status characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Social Status") & (n_data["ScenarioTypeStrict"]=="Social Status"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Social Status") & (N_data["ScenarioTypeStrict"]=="Social Status"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Social Status'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Social Status'})

        # create dummy variable to estimate the preference for sparing high status characters
        n_X = (n_dd.loc[:,"Social Status"]=="High").astype(int)
        N_X = (N_dd.loc[:,"Social Status"]=="High").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd.loc[:,"Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd.loc[:,y].to_numpy()          # predicted outcomes
        n_weights = n_dd.loc[:,"weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()                # predicted outcomes
        N_weights = N_dd.loc[:,"weights"].to_numpy()    # define weights


    # calculate point estimate
    beta_ppi = ppi_ols_pointestimate(X=n_X, Y=n_Y_human, Yhat=n_Y_silicon, 
                                     X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                     w=n_weights, w_unlabeled=N_weights)
    
    # using ppi function to calculate point estimates (lambda=0)
    beta_hum = ppi_ols_pointestimate(X=n_X, Y=n_Y_human, Yhat=n_Y_silicon, 
                                     X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                     w=n_weights, w_unlabeled=N_weights, 
                                     lam=0)
    
    beta_sil = ppi_ols_pointestimate(X=N_X, Y=N_Y_silicon, Yhat=N_Y_silicon, 
                                     X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                     w=N_weights, w_unlabeled=N_weights, 
                                     lam=0)
    
    # using statsmodels to calculate point estimates (same results as with PPI)
    beta_hum_sm = sm.WLS(endog=n_Y_human, exog=n_X, weights=n_weights).fit().params[1]
    beta_sil_sm = sm.WLS(endog=N_Y_silicon, exog=N_X, weights=N_weights).fit().params[1]

    # calculate confidence intervals for PPI, human subjects, and silicon subjects
    lower_CI_ppi, upper_CI_ppi = ppi_ols_ci(X=n_X, Y=n_Y_human, Yhat=n_Y_silicon, 
                                            X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                            w=n_weights, w_unlabeled=N_weights, alpha=alpha)
    
    lower_CI_hum, upper_CI_hum = classical_ols_ci(X=n_X, Y=n_Y_human, w=n_weights, alpha=alpha)

    lower_CI_sil, upper_CI_sil = classical_ols_ci(X=N_X, Y=N_Y_silicon, w=N_weights, alpha=alpha)


    # zscore for two tailed test
    z = stats.norm.ppf(0.975)
    
    # calculate standard errors for PPI, human subjects, and silicon subjects
    se_ppi = (upper_CI_ppi[1] - lower_CI_ppi[1]) / (2 * z)
    
    se_hum = (upper_CI_hum[1] - lower_CI_hum[1]) / (2 * z)

    se_sil = (upper_CI_sil[1] - lower_CI_sil[1]) / (2 * z)
    

    # calculate rho
    beta = sm.WLS(n_Y_human, n_X, weights=n_weights).fit().params

    grads, grads_hat, grads_hat_unlabeled, inv_hessian = _ols_get_stats(
        pointest=beta, 
        X=n_X,
        Y=n_Y_human,
        Yhat= n_Y_silicon,
        X_unlabeled=N_X,
        Yhat_unlabeled=N_Y_silicon,
        w=n_weights,
        w_unlabeled=N_weights,
        use_unlabeled=False)
    
    rho_sq, var_y = _power_analysis_stats(grads, grads_hat, inv_hessian)

    # create and return the output DataFrame
    output_df = pd.DataFrame({
        "y": y,                              
        "x": x,                               # Predictor variable (scenario attribute)
        "beta_ppi": beta_ppi[1],              # PPI point estimate
        "beta_hum": beta_hum[1],              # Human subjects point estimate
        "beta_hum_sm": beta_hum_sm,           # Human subjects point estimate (statsmodels)
        "beta_sil": beta_sil[1],              # Silicon subjects point estimate
        "beta_sil_sm": beta_sil_sm,           # Silicon subjects point estimate (statsmodels)
        "se_ppi": se_ppi,                     # PPI standard error
        "se_hum": se_hum,                     # Human subjects standard error
        "se_sil": se_sil,                     # Silicon subjects standard error
        "lower_ppi": lower_CI_ppi[1],         # The lower bound of the PPI confidence interval
        "upper_ppi": upper_CI_ppi[1],         # The upper bound of the PPI confidence interval
        "lower_hum": lower_CI_hum[1],         # The lower bound of the human subjects confidence interval
        "upper_hum": upper_CI_hum[1],         # The upper bound of the human subjects confidence interval
        "lower_sil": lower_CI_sil[1],         # The lower bound of the silicon subjects confidence interval
        "upper_sil": upper_CI_sil[1],         # The upper bound of the silicon subjects confidence interval
        "ppi_corr": np.sqrt(rho_sq[1])},      # The association between predictions and outcomes
        index=[0])
    
    return output_df 

In [282]:
ids = df["ResponseID"].unique()
n = 22000
N = len(ids) - n
random.seed(2024)

n_ids = random.sample(ids.tolist(), k=n)
N_ids = random.sample(list(set(ids) - set(n_ids)), k=N)

df_human = df[ df["ResponseID"].isin(n_ids) ]
df_silicon = df [ df["ResponseID"].isin(N_ids)]

models = ["gpt4turbo_wp_Saved_1"]

results2 = pd.DataFrame()
for model in models: 
    
    print("Model: ", model)
    results1 = pd.concat([
        compute_amce_ppi(df_human, df_silicon, x="Intervention", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Barrier", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Gender", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Fitness", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Social Status", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="CrossingSignal",y=model),
        compute_amce_ppi(df_human, df_silicon, x="Age", y=model),
        compute_amce_ppi(df_human, df_silicon, x="Utilitarian", y=model),
        compute_amce_ppi(df_human, df_silicon, x="Species", y=model)
    ],ignore_index=True)
    
    results2 = pd.concat([results2, results1],ignore_index=True).sort_values(by=["y","ppi_corr"], ascending=False)
    
results2.to_csv("../Data/7_rho.csv", index=False)
results2

Model:  gpt4turbo_wp_Saved_1


,y,x,beta_ppi,beta_hum,beta_hum_sm,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,lower_ppi,upper_ppi,lower_hum,upper_hum,lower_sil,upper_sil,ppi_corr
0,gpt4turbo_wp_Saved_1,Intervention,0.088327,0.087635,0.087635,0.085495,0.085495,0.005254,0.005589,0.001077,0.078031,0.098625,0.076681,0.098589,0.083385,0.087606,0.347161
1,gpt4turbo_wp_Saved_1,Barrier,0.098643,0.094870,0.094870,0.486976,0.486976,0.008014,0.008437,0.001407,0.082938,0.114351,0.078333,0.111406,0.484219,0.489734,0.322217
5,gpt4turbo_wp_Saved_1,CrossingSignal,0.376197,0.385165,0.385165,0.656098,0.656098,0.008870,0.009329,0.001525,0.358815,0.393586,0.366881,0.403450,0.653109,0.659087,0.316794
3,gpt4turbo_wp_Saved_1,Fitness,0.177001,0.181841,0.181841,0.021571,0.021571,0.013119,0.013586,0.002594,0.151250,0.202674,0.155213,0.208468,0.016487,0.026655,0.265618
2,gpt4turbo_wp_Saved_1,Gender,0.120163,0.120048,0.120048,0.201523,0.201523,0.012625,0.012988,0.002448,0.095419,0.144908,0.094592,0.145504,0.196725,0.206322,0.239029
6,gpt4turbo_wp_Saved_1,Age,0.518358,0.513868,0.513868,0.181983,0.181983,0.011447,0.011708,0.002503,0.495850,0.540722,0.490921,0.536815,0.177077,0.186888,0.219661
4,gpt4turbo_wp_Saved_1,Social Status,0.236123,0.240562,0.240562,0.040554,0.040554,0.036750,0.037505,0.006971,0.164071,0.308130,0.167054,0.314070,0.026890,0.054217,0.204905
7,gpt4turbo_wp_Saved_1,Utilitarian,0.557395,0.558782,0.558782,0.554648,0.554648,0.010660,0.010860,0.002100,0.536496,0.578281,0.537496,0.580067,0.550533,0.558763,0.199685
8,gpt4turbo_wp_Saved_1,Species,0.671840,0.672469,0.672469,0.844214,0.844214,0.009696,0.009703,0.001450,0.652837,0.690846,0.653452,0.691486,0.841372,0.847055,0.040058


Next, we vary the number of human subjects and silicon subjects in a simulation.

In [259]:
# sample size of human subjects
ns = [500,750]
ns= [500]

# multiples of human subjects sample size
ks = list([0.1, 0.25, 0.5, 0.75]) + list(np.arange(1, 10.5, 0.5))

# number of repetitions for combinations of n and N
reps = 50

# LLM predictions
Ys = models
Ys = ["gpt4turbo_wp_Saved_1"]

# structural attributes of scenarios
Xs_structural  = ['Intervention', 'Barrier','CrossingSignal']

# attributes of characters
Xs_characters = ['Gender','Fitness','Social Status','Age','Utilitarian','Species']

# all attributes
Xs = Xs_structural + Xs_characters

result = pd.DataFrame()

# loop models
for y in Ys:
  
  print(f"Iterating predictions from the model: {y}")
  
  # loop over predictors
  for x in Xs:
    print(f"    Predictor: {x}")

    # loop over sample sizes of human subjects
    for n in ns:
      print(f"        Human sample size: {n}")

      # sample size silicon subjects 
      Ns = [int(n * k) for n in ns for k in ks]
      
      # loop over sample sizes of silicon subjects
      for N in Ns:
        
        # loop over repetitions
        for r in range(reps):

          # subset to dilemmas with variation on structural attribute
          if x in Xs_structural:

              cnt = df.groupby("ResponseID")[x].nunique()
              ids = cnt[ cnt > 1].index.tolist()

          # subset to dilemmas with relevant character attribute
          if x in Xs_characters:

              ids = df.loc[ (df["ScenarioType"]==x) & (df["ScenarioTypeStrict"]==x), "ResponseID"].tolist()
          
          # skip current iteration if target n is larger than population
          if (len(ids) < n):
             continue 

          # sample dilemmas for human subjects sample
          n_ids = random.sample(ids, k=n)
          
          # get remaining dilemma ids to sample from
          remaining_ids = list(set(ids) - set(n_ids))

          # skip current iteration if target N is larger than population
          if (len(remaining_ids) < N):
             continue 
          
          # sample dilemmas for silicon subjects sample
          N_ids = random.sample(remaining_ids, k=N)

          # subset data
          df_human = df[ df["ResponseID"].isin(n_ids) ]
          df_silicon = df [ df["ResponseID"].isin(N_ids)]

          # compute acme on n human subjects and N silicon subjects
          ppi = compute_amce_ppi(n_data=df_human, N_data=df_silicon, x=x, y=y)

          # store data
          ppi["n"] = n
          ppi["N"] = N
          
          result = pd.concat([result, ppi], ignore_index=True)
          


Iterating predictions from the model: gpt4turbo_wp_Saved_1
    Predictor: Intervention
        Human sample size: 500
    Predictor: Barrier
        Human sample size: 500
    Predictor: CrossingSignal
        Human sample size: 500
    Predictor: Gender
        Human sample size: 500
    Predictor: Fitness
        Human sample size: 500
    Predictor: Social Status
        Human sample size: 500
    Predictor: Age
        Human sample size: 500
    Predictor: Utilitarian
        Human sample size: 500
    Predictor: Species
        Human sample size: 500


We benchmark the silicon subjects design and the mixed subjects design against a human subjects approach.

In [260]:
# subset point estimates of AMCEs from the entire human subjects sample
benchmark = amce_human_subjects.loc[:, ['x', 'beta']].rename(columns={'beta': 'param'})

# merge benchmark with results from simulation
result_wb = pd.merge(result, benchmark, on='x', how='left')

# report if true value is within the confidence interval from the mixed subjects 
result_wb['coverage_ppi'] = (
    (result_wb['lower_ppi'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_ppi'])
).astype(int) 

# report if true value is within the confidence interval from the silicon subjects 
result_wb['coverage_sil'] = (
    (result_wb['lower_sil'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_sil'])
).astype(int) 

# report if true value is within the confidence interval from the silicon subjects 
result_wb['coverage_hum'] = (
    (result_wb['lower_hum'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_hum'])
).astype(int) 

result_wb

,y,x,beta_ppi,beta_hum,beta_hum_sm,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,...,upper_hum,lower_sil,upper_sil,ppi_corr,n,N,param,coverage_ppi,coverage_sil,coverage_hum
0,gpt4turbo_wp_Saved_1,Intervention,0.058802,0.054994,0.054994,0.222829,0.222829,0.036640,0.036926,0.106417,...,0.127368,0.014255,0.431403,0.403999,500,50,0.065392,1,1,1
1,gpt4turbo_wp_Saved_1,Intervention,0.081533,0.086763,0.086763,0.016593,0.016593,0.036614,0.036842,0.109100,...,0.158973,-0.197238,0.230424,0.365939,500,50,0.065392,1,1,1
2,gpt4turbo_wp_Saved_1,Intervention,0.118200,0.113304,0.113304,0.206762,0.206762,0.036811,0.037042,0.108244,...,0.185905,-0.005393,0.418916,0.342911,500,50,0.065392,1,1,1
3,gpt4turbo_wp_Saved_1,Intervention,0.121787,0.124721,0.124721,0.006683,0.006683,0.036348,0.036486,0.116379,...,0.196233,-0.221415,0.234782,0.300395,500,50,0.065392,1,1,1
4,gpt4turbo_wp_Saved_1,Intervention,0.080476,0.075860,0.075860,0.129394,0.129394,0.036433,0.036719,0.115225,...,0.147829,-0.096443,0.355232,0.429904,500,50,0.065392,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7645,gpt4turbo_wp_Saved_1,Species,0.610867,0.612326,0.612326,0.838337,0.838337,0.030345,0.030329,0.007954,...,0.671770,0.822749,0.853926,0.074048,500,3500,0.632202,1,0,1
7646,gpt4turbo_wp_Saved_1,Species,0.660920,0.657357,0.657357,0.846128,0.846128,0.028433,0.028550,0.007795,...,0.713314,0.830849,0.861407,0.084042,500,3500,0.632202,1,0,1
7647,gpt4turbo_wp_Saved_1,Species,0.589789,0.589789,0.589789,0.845596,0.845596,0.031758,0.031742,0.007782,...,0.652002,0.830344,0.860848,0.023384,500,3500,0.632202,1,0,1
7648,gpt4turbo_wp_Saved_1,Species,0.626570,0.623711,0.623711,0.848598,0.848598,0.030143,0.030161,0.007695,...,0.682826,0.833516,0.863681,0.049116,500,3500,0.632202,1,0,1


In [261]:
# Group by n, N, and LLM then calculate mean across repetitions
stats = ['beta_ppi','se_ppi','lower_ppi','upper_ppi','coverage_ppi','ppi_corr',
         'beta_sil','se_sil','lower_sil','upper_sil','coverage_sil',
         'beta_hum','se_hum','lower_hum','upper_hum','coverage_hum']

summ = result_wb.groupby(['n','N','y','x','param'])[stats].mean().reset_index()

# Calculate bias columns
summ['repetitions'] = reps
summ['bias_ppi'] = summ['beta_ppi'] - summ['param']
summ['bias_sil'] = summ['beta_sil'] - summ['param']
summ['bias_hum'] = summ['beta_hum'] - summ['param']

summ['rmse_ppi'] = np.sqrt(summ['bias_ppi']**2 + summ['se_ppi']**2)
summ['rmse_sil'] = np.sqrt(summ['bias_sil']**2 + summ['se_sil']**2)
summ['rmse_hum'] = np.sqrt(summ['bias_hum']**2 + summ['se_hum']**2)

# Save averaged simulation results to compressed csv file
summ.to_csv("../Data/7_ResultsPPI.csv.gz", compression="gzip", index=False)
summ

,n,N,y,x,param,beta_ppi,se_ppi,lower_ppi,upper_ppi,coverage_ppi,...,lower_hum,upper_hum,coverage_hum,repetitions,bias_ppi,bias_sil,bias_hum,rmse_ppi,rmse_sil,rmse_hum
0,500,50,gpt4turbo_wp_Saved_1,Age,0.494434,0.498906,0.033000,0.433780,0.563136,0.82,...,0.432631,0.562199,0.80,50,0.004471,-0.309638,0.002980,0.033301,0.328346,0.033188
1,500,50,gpt4turbo_wp_Saved_1,Barrier,0.150271,0.164233,0.041096,0.083722,0.244815,0.92,...,0.082326,0.244227,0.94,50,0.013962,0.351512,0.013006,0.043403,0.367377,0.043301
2,500,50,gpt4turbo_wp_Saved_1,CrossingSignal,0.339410,0.343985,0.046715,0.252480,0.435601,0.82,...,0.251228,0.434949,0.82,50,0.004574,0.345429,0.003678,0.046939,0.362781,0.047012
3,500,50,gpt4turbo_wp_Saved_1,Fitness,0.130361,0.119855,0.037244,0.046906,0.192902,0.82,...,0.047424,0.193992,0.82,50,-0.010507,-0.129311,-0.009653,0.038698,0.169223,0.038617
4,500,50,gpt4turbo_wp_Saved_1,Gender,0.155021,0.144870,0.036773,0.072700,0.216847,0.90,...,0.073309,0.217922,0.90,50,-0.010151,0.070246,-0.009405,0.038148,0.129350,0.038072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,500,4750,gpt4turbo_wp_Saved_1,CrossingSignal,0.339410,0.327705,0.045836,0.237868,0.417541,0.88,...,0.234371,0.420453,0.88,50,-0.011705,0.326303,-0.011998,0.047307,0.326530,0.048964
149,500,4750,gpt4turbo_wp_Saved_1,Intervention,0.065392,0.068602,0.034713,0.000479,0.136552,0.76,...,-0.005902,0.139112,0.80,50,0.003210,0.022771,0.001213,0.034861,0.025611,0.037014
150,500,5000,gpt4turbo_wp_Saved_1,Barrier,0.150271,0.168721,0.039190,0.091976,0.245596,0.92,...,0.085994,0.247469,0.94,50,0.018450,0.331450,0.016461,0.043315,0.331634,0.044361
151,500,5000,gpt4turbo_wp_Saved_1,CrossingSignal,0.339410,0.344361,0.044902,0.256287,0.432301,0.88,...,0.245883,0.429332,0.86,50,0.004951,0.324124,-0.001803,0.045174,0.324343,0.046834
